In [9]:
!pip install yfinance

  Obtaining dependency information for yfinance from https://files.pythonhosted.org/packages/09/82/5a5eee685095a6c1f8968db75bba8536baa8046c970be25e1ac91a843839/yfinance-0.2.33-py2.py3-none-any.whl.metadata
  Obtaining dependency information for frozendict>=2.3.4 from https://files.pythonhosted.org/packages/b6/2c/e391d6d57bbd837d383a6b69b54ecb4768d75b31d8bc7a380b4b4afe4cb2/frozendict-2.3.10-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
      --------------------------------------- 0.0/2.9 MB 991.0 kB/s eta 0:00:03
     --- ------------------------------------ 0.3/2.9 MB 3.2 MB/s eta 0:00:01
     ------- -------------------------------- 0.5/2.9 MB 4.8 MB/s eta 0:00:01
     ------------------ --------------------- 1.4/2.9 MB 7.9 MB/s eta 0:00:01
     ---------------------------- ----------- 2.1/2.9 MB 10.3 MB/s eta 0:00:01
     ---------------------------------------  2.9/2.9 MB 11.1 MB/s eta 0:00:01
     ---------------------------------

In [10]:
# libraries 
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import yfinance as yf

# Analysing Stocks:
### 1. Preparing data.


In [11]:
# reading nifty-50
nifty50 = pd.read_csv("../data/nifty_50.csv")
symbols = nifty50["Symbol"] + ".NS"
symbols = symbols.to_list()

In [12]:
symbols

['ADANIENT.NS',
 'ADANIPORTS.NS',
 'APOLLOHOSP.NS',
 'ASIANPAINT.NS',
 'AXISBANK.NS',
 'BAJAJ-AUTO.NS',
 'BAJFINANCE.NS',
 'BAJAJFINSV.NS',
 'BPCL.NS',
 'BHARTIARTL.NS',
 'BRITANNIA.NS',
 'CIPLA.NS',
 'COALINDIA.NS',
 'DIVISLAB.NS',
 'DRREDDY.NS',
 'EICHERMOT.NS',
 'GRASIM.NS',
 'HCLTECH.NS',
 'HDFCBANK.NS',
 'HDFCLIFE.NS',
 'HEROMOTOCO.NS',
 'HINDALCO.NS',
 'HINDUNILVR.NS',
 'ICICIBANK.NS',
 'INDUSINDBK.NS',
 'INFY.NS',
 'ITC.NS',
 'JSWSTEEL.NS',
 'KOTAKBANK.NS',
 'LT.NS',
 'LTIM.NS',
 'M&M.NS',
 'MARUTI.NS',
 'NESTLEIND.NS',
 'NTPC.NS',
 'ONGC.NS',
 'POWERGRID.NS',
 'RELIANCE.NS',
 'SBILIFE.NS',
 'SBIN.NS',
 'SUNPHARMA.NS',
 'TATAMOTORS.NS',
 'TATASTEEL.NS',
 'TCS.NS',
 'TATACONSUM.NS',
 'TECHM.NS',
 'TITAN.NS',
 'ULTRACEMCO.NS',
 'UPL.NS',
 'WIPRO.NS']

In [13]:
data = yf.download(tickers = symbols, start = '2013-01-01', end = '2023-12-18')
data

[*********************100%%**********************]  50 of 50 completed


Adj Close                                            \
            ADANIENT.NS ADANIPORTS.NS APOLLOHOSP.NS ASIANPAINT.NS   
Date                                                                
2013-01-01    37.013351    129.891815    754.627014    407.069824   
2013-01-02    38.281120    130.504517    749.887817    402.366821   
2013-01-03    37.657299    130.363174    767.887085    404.644775   
2013-01-04    37.120682    129.609055    774.732544    405.132538   
2013-01-07    37.482899    132.012711    771.716614    404.428375   
...                 ...           ...           ...           ...   
2023-12-11  2855.800049   1031.900024   5547.350098   3233.000000   
2023-12-12  2857.649902   1041.949951   5428.950195   3224.750000   
2023-12-13  2875.050049   1063.500000   5426.750000   3243.649902   
2023-12-14  2894.050049   1074.699951   5515.899902   3241.350098   
2023-12-15  2991.800049   1078.550049   5551.200195   3313.899902   

                                                                   \
            AXISBANK.NS BAJAJ-AUTO.NS BAJAJFINSV.NS BAJFINANCE.NS   
Date                                                                
2013-01-01   257.808990   1535.420166     90.304695    126.483383   
2013-01-02   262.377899   1586.087158     90.170959    125.093628   
2013-01-03   262.964417   1595.198120     90.626633    128.461517   
2013-01-04   260.883331   1588.500122     92.181801    128.521759   
2013-01-07   259.729279   1582.233887     90.864357    128.331772   
...                 ...           ...           ...           ...   
2023-12-11  1116.500000   6138.399902   1702.800049   7292.750000   
2023-12-12  1131.000000   6254.200195   1706.750000   7314.549805   
2023-12-13  1114.849976   6316.100098   1684.550049   7258.000000   
2023-12-14  1120.449951   6334.850098   1730.949951   7474.200195   
2023-12-15  1120.349976   6275.200195   1733.150024   7515.049805   

                                      ...       Volume                \
           BHARTIARTL.NS     BPCL.NS  ... SUNPHARMA.NS TATACONSUM.NS   
Date                                  ...                              
2013-01-01    279.236420   68.072670  ...       591988       3237577   
2013-01-02    278.326233   70.011871  ...      1043632       7201391   
2013-01-03    284.351288   69.973671  ...      1040878       4841015   
2013-01-04    283.484406   71.454338  ...      1031736       1901621   
2013-01-07    283.311005   73.651474  ...      1968642       2340504   
...                  ...         ...  ...          ...           ...   
2023-12-11   1000.849976  444.500000  ...      1172972        476174   
2023-12-12    995.250000  447.549988  ...      2832873        732063   
2023-12-13    999.900024  447.500000  ...      2448178        899820   
2023-12-14   1005.200012  445.700012  ...      2341190       1980762   
2023-12-15    992.549988  449.000000  ...      3474624       1155502   

                                                                  \
           TATAMOTORS.NS TATASTEEL.NS   TCS.NS TECHM.NS TITAN.NS   
Date                                                               
2013-01-01       3980484     32584167   509580   543928  1161263   
2013-01-02       6929285     27256503  1880246  1199844  1252267   
2013-01-03       6165017     34469672  2635130  1138816  1771023   
2013-01-04       8965650     38258536  5228760  1099620  1275766   
2013-01-07       4943021     71920525  1820752   989776  1700272   
...                  ...          ...      ...      ...      ...   
2023-12-11       6403209     24038059  1102503  1558876   620604   
2023-12-12       6461539     30696410  2236866  1114846  1075941   
2023-12-13       5807736     22767972  3850968  3025706   717618   
2023-12-14       6732352     43372143  3290396  5287733   992762   
2023-12-15      12082079     77377446  8531230  5900381   906271   

                                             
           ULTRACEMCO.NS   UPL.NS  WIPRO.NS  
Date                      

In [14]:
data.stack()

Adj Close         Close          High  \
Date                                                                 
2013-01-01 ADANIENT.NS       37.013351     42.288589     42.526165   
           ADANIPORTS.NS    129.891815    137.800003    138.149994   
           APOLLOHOSP.NS    754.627014    788.200012    795.700012   
           ASIANPAINT.NS    407.069824    442.304993    445.480011   
           AXISBANK.NS      257.808990    272.540009    274.299988   
...                                ...           ...           ...   
2023-12-15 TECHM.NS        1306.099976   1306.099976   1324.800049   
           TITAN.NS        3600.550049   3600.550049   3615.000000   
           ULTRACEMCO.NS  10029.450195  10029.450195  10059.500000   
           UPL.NS           610.849976    610.849976    612.799988   
           WIPRO.NS         446.549988    446.549988    449.500000   

                                  Low          Open      Volume  
Date                                                             
2013-01-01 ADANIENT.NS      41.813435     41.890072   6193245.0  
           ADANIPORTS.NS   135.300003    136.250000    353823.0  
           APOLLOHOSP.NS   786.000000    788.900024     49806.0  
           ASIANPAINT.NS   441.000000    444.500000    299620.0  
           AXISBANK.NS     272.100006    273.940002   3343610.0  
...                               ...           ...         ...  
2023-12-15 TECHM.NS       1262.500000   1270.000000   5900381.0  
           TITAN.NS       3584.800049   3615.000000    906271.0  
           ULTRACEMCO.NS  9900.650391  10000.000000    439516.0  
           UPL.NS          601.450012    601.450012   4941179.0  
           WIPRO.NS        433.500000    436.049988  20783617.0  

[131954 rows x 6 columns]